In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
real_test = pd.read_csv("labhourformattest_72.csv")
real_train = pd.read_csv("labhourformatrain_72.csv")
timegan = pd.read_csv("labtimeGAN-generated.csv")
deepecho_100 = pd.read_csv("deepEchoGenerated-100epoch.csv")

In [7]:
testids = list(set(timegan["hadm_id"]))[:200]
timegan_test = timegan[timegan["hadm_id"].isin(testids)]
timegan_train = timegan[~timegan["hadm_id"].isin(testids)]

In [8]:
deepecho_100= deepecho_100.loc[:, ~deepecho_100.columns.isin(['gender', 'anchor_age',"gender", "insurance","marital_status","race","index","hour"])]


In [10]:
len(set(real_test["hadm_id"])), len(set(real_train["hadm_id"])), len(set(timegan_test["hadm_id"])), len(set(deepecho_100["hadm_id"]))


(200, 721, 200, 200)

In [27]:
from scipy.spatial import distance
import statistics

In [ ]:
# want one distance for each column 

In [26]:
distance.hamming([1, 0, 0], [0, 1, 0])

0.6666666666666666

In [70]:
def gethammingdistance(data1, data2, columns):
    data1ids = list(set(data1["hadm_id"]))
    data2ids = list(set(data2["hadm_id"]))
    hammingdistmean = {}
    hammingdistmedian = {}
    # get mean, median hamming distance for each column
    for c in columns:
        cdistances = {}
        distances = []
        for i in data1ids[:200]:
            data1seq = data1[data1["hadm_id"]==i][c].to_list()
            for j in data2ids:
                data2seq = data2[data2["hadm_id"]==j][c].to_list()
                distances.append(distance.hamming(data1seq, data2seq))
        
        hammingdistmean[c] = statistics.mean(distances)    
        hammingdistmedian[c] = statistics.median(distances)
        print("done column ", c)
    return hammingdistmean, hammingdistmedian
            

In [71]:
columns = ["magnesium","lactate","creatinine","CBC","calcium","Urine","ABG","leukocytes","anion gap","pt","urea nitrogen","bicarbonate","Electrolytes"]
hammingdistmean, hammingdistmedian = gethammingdistance(real_train, real_test, columns)

done column  magnesium
done column  lactate
done column  creatinine
done column  CBC
done column  calcium
done column  Urine
done column  ABG
done column  leukocytes
done column  anion gap
done column  pt
done column  urea nitrogen
done column  bicarbonate
done column  Electrolytes


In [73]:
hammingdistmeandeep, hammingdistmediandeep = gethammingdistance(real_train, deepecho_100, columns)

done column  magnesium
done column  lactate
done column  creatinine
done column  CBC
done column  calcium
done column  Urine
done column  ABG
done column  leukocytes
done column  anion gap
done column  pt
done column  urea nitrogen
done column  bicarbonate
done column  Electrolytes


In [74]:
hammingdistmeantime, hammingdistmediantime = gethammingdistance(real_train, timegan_test, columns)

done column  magnesium
done column  lactate
done column  creatinine
done column  CBC
done column  calcium
done column  Urine
done column  ABG
done column  leukocytes
done column  anion gap
done column  pt
done column  urea nitrogen
done column  bicarbonate
done column  Electrolytes


In [79]:
hammingmeandic = {}
hammingmeandic["real"]  = list(hammingdistmean.values()) + [statistics.mean(list(hammingdistmean.values()))]
hammingmeandic["deepEcho"]  = list(hammingdistmeandeep.values()) + [statistics.mean(list(hammingdistmeandeep.values()))]
hammingmeandic["timeGAN"]  = list(hammingdistmeantime.values()) + [statistics.mean(list(hammingdistmeantime.values()))]

In [80]:
hammingmediandic = {}
hammingmediandic["real"]  = list(hammingdistmedian.values()) + [statistics.median(list(hammingdistmedian.values()))]
hammingmediandic["deepEcho"]  = list(hammingdistmediandeep.values())+ [statistics.median(list(hammingdistmediandeep.values()))]
hammingmediandic["timeGAN"]  = list(hammingdistmediantime.values())+ [statistics.median(list(hammingdistmediantime.values()))]

# hamming distance

In [77]:
print("Mean Hamming Distance")
pd.DataFrame.from_dict(hammingmeandic, orient='index', columns=columns+["total"])

Mean Hamming Distance


,magnesium,lactate,creatinine,CBC,calcium,Urine,ABG,leukocytes,anion gap,pt,urea nitrogen,bicarbonate,Electrolytes,total
real,0.084478,0.084160,0.104409,0.187115,0.154560,0.224317,0.171658,0.011763,0.092256,0.068203,0.103803,0.097790,0.213907,0.122955
deepEcho,0.091347,0.090913,0.115204,0.205478,0.168711,0.243676,0.204838,0.034437,0.100638,0.080507,0.114898,0.104556,0.237922,0.137933
timeGAN,0.046111,0.052778,0.058403,0.131459,0.101942,0.158449,0.126323,0.006875,0.051042,0.041736,0.057986,0.054236,0.149126,0.079728


In [78]:
print("Median Hamming Distance")
pd.DataFrame.from_dict(hammingmediandic, orient='index', columns=columns+["total"])

Median Hamming Distance


,magnesium,lactate,creatinine,CBC,calcium,Urine,ABG,leukocytes,anion gap,pt,urea nitrogen,bicarbonate,Electrolytes,total
real,0.083333,0.069444,0.111111,0.180556,0.152778,0.208333,0.152778,0.013889,0.097222,0.069444,0.111111,0.097222,0.208333,0.111111
deepEcho,0.097222,0.083333,0.111111,0.208333,0.166667,0.236111,0.194444,0.027778,0.097222,0.083333,0.111111,0.097222,0.236111,0.111111
timeGAN,0.041667,0.041667,0.055556,0.125000,0.097222,0.152778,0.111111,0.000000,0.055556,0.041667,0.055556,0.055556,0.138889,0.055556
